In [6]:
# ADMM adaptive lasso using ProximalOperators with line and golden section search
# Automatic tuning of learning rate and regularization parameter
# This code only only changes the ratio between test data and train data, and changes the input file
# The original source code from https://github.com/patwa67/AUTALASSO

using ProximalOperators
using DelimitedFiles
using Statistics
using LinearAlgebra

In [7]:
# Function for Golden section search to optimize lambda
function gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)
  lama =alam*maxnorm # Lower lambda
  lamb =blam*maxnorm # Higher lambda
  gr = (sqrt(5.0) + 1.0) / 2.0 # Golden section ratio
  toladmm = 1e-4 # Convergence tolerance for ADMM
  fc = lasso_admm(Xtrainhot, ytrain, lama, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
  fd = lasso_admm(Xtrainhot, ytrain, lamb, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
  iter = 2
  meanlam = zero(1.0:100.0)
  #meanlam[iter] = (lama+lamb)/2
  meanloss = zero(1.0:100.0)
  #meanloss[1] = max(lossc,lossd)
  #meanloss[iter] = (lossc+lossd)/2
  lamc = lamb - (lamb - lama) / gr
  lamd = lama + (lamb - lama) / gr
  println("lossc =$lossc")
  println("lossd =$lossd")
  println("lambdaa =$lama")
  println("lambdab =$lamb")
  println("lambdac =$lamc")
  println("lambdad =$lamd")
  iter = 2
  nodrun = 0
  while abs(lamc - lamd)/((lamc + lamd)/2.0) > tolgss # Run GSS until convergence
    iter = iter+1
    if iter == 3
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    else
    if nodrun==1
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    else
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    end
    end
    meanlam[iter] = (lamc+lamd)/2.0
    meanloss[iter] = (lossc+lossd)/2.0
    # Stop GSS if test MSE is increased two consecutive iterations
    if (meanloss[iter] > meanloss[iter-1])&&(meanloss[iter-1] > meanloss[iter-2])
      break
    end
    if lossc < lossd
      lamb = lamd
      fd=fc
      lossd=lossc
      nodrun=1
    else
      lama = lamc
      fc=fd
      lossc=lossd
      nodrun=0
    end
    lamc = lamb - (lamb - lama) / gr
    lamd = lama + (lamb - lama) / gr
    #println("lossc =$lossc")
    #println("lossd =$lossd")
    println("lambdac =$lamc")
    println("lambdad =$lamd")
  end
  # Final ADMM for optimized lambda
  lamopt = meanlam[iter-2]
  fmean1 = (fc[1]+fd[1])/2.0
  fmean2 = (fc[2]+fd[2])/2.0
  fopt = lasso_admm(Xtrainhot, ytrain, lamopt, fmean1,fmean2,f, abscovinv,toladmm)
  lossopt= 0.5*norm(Xtesthot*fopt[1].-ytest)^2
  acc = cor(Xtesthot*fopt[1],ytest)

  return lamopt,fopt,lossopt,acc
end

gss_opt (generic function with 1 method)

In [8]:
# Function for Proximal ADMM lasso with line search
function lasso_admm(Xtrainhot, ytrain, lam, theta, beta, f, abscovinv,tol; maxit=5000)
  u = zero(theta)
  grad = zero(theta)
  lamw = lam*abscovinv # Regularization parameter times weights
  g = NormL1(lamw) # Regularization function
  c = 0.5
  lr = 1.0
  loss(theta) = 0.5*norm(Xtrainhot*theta-ytrain)^2 # Loss function for line search
  for it = 1:maxit
    # Line search
    it % 8 == 1 && (grad = Xtrainhot'*(Xtrainhot*beta-ytrain))
    while  it % 20 == 2 && loss(theta) > (loss(beta) + grad'*(-beta) + (1.0/(2.0*lr))*norm(-beta)^2)
      lr = lr * c
      #println(lr)
    end
    gam = lr
    # ADMM perform f-update step
    prox!(beta, f, theta - u, gam)
    # ADMM perform g-update step
    prox!(theta, g, beta + u, gam)
    # Stopping criterion for ADMMM
    if norm(beta-theta, Inf) <= tol*(1+norm(u, Inf))
      break
    end
    # Dual update
    u .+= beta - theta
    #print(it)
  end
  return theta,beta,tol
end

lasso_admm (generic function with 1 method)

In [9]:
data_file_path = "/Users/dong/GWAS/AtPolyDB/Silique_16.csv"

"/Users/dong/GWAS/AtPolyDB/Silique_16.csv"

In [10]:
# Read Silique_16.csv, and partition into train and test parts
X = readdlm(data_file_path,',')
ytot = (X[:,1].-mean(X[:,1])) # Center y to mean zero
ytrain = ytot[1:76]
Xtest= X[77:size(X)[1],2:size(X)[2]]
ytest = ytot[77:size(X)[1]]
Xtrain = X[1:76,2:size(X)[2]]

76×214051 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  …  0.0  0.0  0.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  2.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0  …  2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  2.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  2.0  2.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  2.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [11]:
# One hot encoding training data
Xtrain0 = copy(Xtrain)
Xtrain1 = copy(Xtrain)
Xtrain2 = copy(Xtrain)
Xtrain0[Xtrain0.==1] .= 2
Xtrain0[Xtrain0.==0] .= 1
Xtrain0[Xtrain0.==2] .= 0
Xtrain1[Xtrain1.==2] .= 0
Xtrain2[Xtrain2.==1] .= 0
Xtrain2[Xtrain2.==2] .= 1
Xtrainhot = hcat(Xtrain0,Xtrain1,Xtrain2)
# Set unimportant allocations to zero
Xtrain0 = 0
Xtrain1 = 0
Xtrain2 = 0
Xtrain = 0

0

In [12]:
# One hot encoding test data
Xtest0 = copy(Xtest)
Xtest1 = copy(Xtest)
Xtest2 = copy(Xtest)
Xtest0[Xtest0.==1] .= 2
Xtest0[Xtest0.==0] .= 1
Xtest0[Xtest0.==2] .= 0
Xtest1[Xtest1.==2] .= 0
Xtest2[Xtest2.==1] .= 0
Xtest2[Xtest2.==2] .= 1
Xtesthot = hcat(Xtest0,Xtest1,Xtest2)
# Set unimportant allocations to zero
Xtest0 = 0
Xtest1 = 0
Xtest2 = 0
Xtest = 0

0

In [13]:
# Factor for initial lower lambda
alam = 0.0001
# Factor for initial upper lambda
blam = 1.0
# Convergence factor for lambda in gss_opt
tolgss = 0.01
# Find lambda where all reg coeff are zero
maxnorm = norm(Xtrainhot'*ytrain, Inf)
# The least squares loss function
f = LeastSquares(Xtrainhot, ytrain)
# Inverse covariances to be used as weights in the adaptive lasso
abscovinv = 1.0./abs.(cov(Xtrainhot,ytrain))

642153×1 Matrix{Float64}:
   91.93548387096774
   92.98531810766718
   91.93548387096774
  191.27516778523483
   85.84337349397589
  141.43920595533507
 2714.285714285719
  245.68965517241375
  600.0000000000007
  740.2597402597394
  211.11111111111103
  600.0000000000007
  211.11111111111103
    ⋮
 1239.1304347826017
   65.36697247706418
  410.07194244604307
  814.2857142857157
  196.55172413793096
  256.7567567567567
 1295.4545454545498
  814.2857142857157
  275.3623188405796
  118.99791231732779
   62.84454244762953
   88.92355694227771

In [14]:
# Run AUTALASSO with timing
@time res = gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)


lossc =0.1769368421052631
lossd =0.1769368421052631
lambdaa =0.0003688421052631583
lambdab =3.688421052631583
lambdac =1.4090794342421349
lambdad =2.2797104604947114
lambdac =2.279710460494712
lambdad =2.817790026379006
lambdac =2.817790026379006
lambdad =3.1503414867472888
lambdac =3.1503414867472888
lambdad =3.355869592263301
lambdac =3.355869592263301
lambdad =3.482892947115571
lambdac =3.482892947115571
lambdad =3.561397697779313
lambdac =3.561397697779313
lambdad =3.609916301967841
lambdac =3.609916301967841
lambdad =3.639902448443055
  3.777079 seconds (1.43 M allocations: 586.124 MiB, 33.22% compilation time)


(3.4193812696894357, ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1.556464340782271e-5, -1.69124077244176e-5, -1.556464340782271e-5, 1.0156772245251133e-5, 1.8216660362835313e-5, 6.764334873343714e-6, 1.1104674841888595e-6, -3.220334643727618e-6, -6.58922112828364e-6, 2.6968264231763683e-6  …  -2.8960097557817477e-6, 2.3875042930998625e-6, 4.50229847159811e-6, -2.3837855992470125e-6, 3.7396410750289988e-6, 2.3875042930998625e-6, 2.7213809125070654e-6, -3.798971855206723e-6, -4.93182471705822e-6, 3.8146019367690798e-6], 0.0001), 0.1769368421052631, NaN)

In [15]:
output_dir = "/Users/dong/GWAS/Autalasso/" 


"/Users/dong/GWAS/Autalasso/"

In [16]:
# Save regression coefficients, lambda, MSE and ACC to text files
#writedlm("outbetaQTLMAS.txt", res[2][1])
#writedlm("outlambdaQTLMAS.txt", res[1])
#writedlm("outMSEQTLMAS.txt", res[3]/length(ytest)*2)
#writedlm("outACCQTLMAS.txt", res[4])

writedlm(joinpath(output_dir, "outbeta_Silique16.txt"), res[2][1])
writedlm(joinpath(output_dir, "outlambda_Silique16.txt"), res[1])
writedlm(joinpath(output_dir, "outMSE_Silique16.txt"), res[3]/length(ytest)*2)
writedlm(joinpath(output_dir, "outACC_Silique16.txt"), res[4])

In [17]:
# Predict observations for test data
ytesthat = Xtesthot*res[2][1]

19-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0